# NanoChat Training on Google Colab 🚀

This notebook runs NanoChat training on Google Colab's A100 GPU.

## ⚠️ Important Notes
- **Limited training** for demo/learning purposes
- Full training would take days/weeks
- Model quality will be lower than production
- Colab has time limits (12-24 hours max)

## Setup Steps
1. Change runtime: Runtime → Change runtime type → GPU → A100
2. Run all cells sequentially
3. Choose your config (demo/small/medium)

**Recommended**: Start with `demo` config first!

## Step 1: Check GPU and Install Dependencies

In [ ]:
# Check GPU
import torch
print("=" * 60)
print("GPU INFORMATION")
print("=" * 60)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️  No GPU detected! Please change runtime to GPU.")
print("=" * 60)

In [ ]:
# Install uv (package manager)
!curl -LsSf https://astral.sh/uv/install.sh | sh
import os
os.environ['PATH'] = os.path.expanduser('~/.cargo/bin') + ':' + os.environ.get('PATH', '')

In [ ]:
# Clone nanochat repository
!git clone https://github.com/ideaweaver-ai/nanochat.git
%cd nanochat

In [ ]:
# Setup Python environment
import os
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['NANOCHAT_BASE_DIR'] = '/content/.cache/nanochat'

# Create venv and install dependencies
!uv venv
!uv sync --extra gpu

# Activate venv (for subsequent cells)
import sys
sys.path.insert(0, '/content/nanochat/.venv/lib/python3.10/site-packages')

## Step 2: Choose Configuration

In [ ]:
# Choose your config: 'demo', 'small', or 'medium'
CONFIG = 'demo'  # Change this: demo|small|medium

# Configuration settings
if CONFIG == 'demo':
    DEPTH = 8
    MAX_SEQ_LEN = 512
    DEVICE_BATCH_SIZE = 4
    TOTAL_BATCH_SIZE = 8192
    BASE_ITERATIONS = 500
    MID_ITERATIONS = 200
    SFT_ITERATIONS = 100
    DATA_SHARDS = 10
    print("✅ DEMO MODE: ~1 hour total training")
elif CONFIG == 'small':
    DEPTH = 10
    MAX_SEQ_LEN = 1024
    DEVICE_BATCH_SIZE = 4
    TOTAL_BATCH_SIZE = 16384
    BASE_ITERATIONS = 2000
    MID_ITERATIONS = 500
    SFT_ITERATIONS = 200
    DATA_SHARDS = 20
    print("✅ SMALL MODE: ~3-6 hours total training")
elif CONFIG == 'medium':
    DEPTH = 12
    MAX_SEQ_LEN = 1024
    DEVICE_BATCH_SIZE = 4
    TOTAL_BATCH_SIZE = 32768
    BASE_ITERATIONS = 5000
    MID_ITERATIONS = 1000
    SFT_ITERATIONS = 500
    DATA_SHARDS = 40
    print("✅ MEDIUM MODE: ~9-13 hours total training")
else:
    raise ValueError(f"Unknown config: {CONFIG}")

print(f"\nConfiguration:")
print(f"  Depth: {DEPTH}")
print(f"  Max seq len: {MAX_SEQ_LEN}")
print(f"  Batch size: {DEVICE_BATCH_SIZE}")
print(f"  Base iterations: {BASE_ITERATIONS}")
print(f"  Mid iterations: {MID_ITERATIONS}")
print(f"  SFT iterations: {SFT_ITERATIONS}")

## Step 3: Setup Tokenizer

In [ ]:
# Install Rust (needed for tokenizer)
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
import os
os.environ['PATH'] = os.path.expanduser('~/.cargo/bin') + ':' + os.environ.get('PATH', '')

In [ ]:
# Build Rust tokenizer
!uv run maturin develop --release --manifest-path rustbpe/Cargo.toml

In [ ]:
# Initialize report
!python -m nanochat.report reset

In [ ]:
# Download minimal data for tokenizer
!python -m nanochat.dataset -n 4

In [ ]:
# Download training data (in background)
import subprocess
import os
os.environ['NANOCHAT_BASE_DIR'] = '/content/.cache/nanochat'
download_process = subprocess.Popen(
    ['python', '-m', 'nanochat.dataset', '-n', str(DATA_SHARDS)],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)
print(f"Started downloading {DATA_SHARDS} data shards in background...")

In [ ]:
# Train tokenizer
!python -m scripts.tok_train --max_chars=500000000

In [ ]:
# Evaluate tokenizer
!python -m scripts.tok_eval

## Step 4: Stage 1 - Base Training

In [ ]:
# Wait for data download to complete
download_process.wait()
print("✅ Data download complete!")

In [ ]:
# Base training with limited iterations
print(f"Starting BASE TRAINING ({BASE_ITERATIONS} iterations)...")
print(f"This will take approximately {BASE_ITERATIONS // 100}-{BASE_ITERATIONS // 50} minutes")

!python -m scripts.base_train \
    --depth={DEPTH} \
    --max_seq_len={MAX_SEQ_LEN} \
    --device_batch_size={DEVICE_BATCH_SIZE} \
    --total_batch_size={TOTAL_BATCH_SIZE} \
    --num_iterations={BASE_ITERATIONS} \
    --target_param_data_ratio=-1 \
    --eval_every=100 \
    --eval_tokens=8192 \
    --core_metric_every=-1 \
    --sample_every=200 \
    --run=dummy

In [ ]:
# Evaluate base model
!python -m scripts.base_loss --device_batch_size={DEVICE_BATCH_SIZE}
!python -m scripts.base_eval --max-per-task=50

## Step 5: Stage 2 - Midtraining

In [ ]:
# Download identity conversations
!curl -L -o /content/.cache/nanochat/identity_conversations.jsonl https://karpathy-public.s3.us-west-2.amazonaws.com/identity_conversations.jsonl

In [ ]:
# Midtraining with limited iterations
print(f"Starting MIDTRAINING ({MID_ITERATIONS} iterations)...")
print(f"This will take approximately {MID_ITERATIONS // 50}-{MID_ITERATIONS // 25} minutes")

!python -m scripts.mid_train \
    --device_batch_size={DEVICE_BATCH_SIZE} \
    --max_seq_len={MAX_SEQ_LEN} \
    --total_batch_size={TOTAL_BATCH_SIZE} \
    --num_iterations={MID_ITERATIONS} \
    --eval_every=50 \
    --eval_tokens=8192 \
    --run=dummy

In [ ]:
# Evaluate midtrained model
!python -m scripts.chat_eval -- -i mid --max-problems=50

## Step 6: Stage 3 - Supervised Fine-Tuning (SFT)

In [ ]:
# Calculate SFT batch size
SFT_BATCH_SIZE = max(DEVICE_BATCH_SIZE // 2, 2)
print(f"SFT batch size: {SFT_BATCH_SIZE}")

In [ ]:
# SFT with limited iterations
print(f"Starting SFT ({SFT_ITERATIONS} iterations)...")
print(f"This will take approximately {SFT_ITERATIONS // 20}-{SFT_ITERATIONS // 10} minutes")

!python -m scripts.chat_sft \
    --device_batch_size={SFT_BATCH_SIZE} \
    --num_iterations={SFT_ITERATIONS} \
    --eval_every=25 \
    --eval_steps=10 \
    --eval_metrics_max_problems=50 \
    --run=dummy

In [ ]:
# Evaluate SFT model
!python -m scripts.chat_eval -- -i sft --max-problems=50

## Step 7: Generate Report

In [ ]:
# Generate final report
!python -m nanochat.report generate

## Step 8: Chat with Your Model!

In [ ]:
# Start web chat interface
# Note: This will start a server - you can access it via the URL shown
# For Colab, you may need to use ngrok or similar for external access
print("To chat with your model, run:")
print("  python -m scripts.chat_web")
print("\nOr use CLI:")
print("  python -m scripts.chat_cli -p 'Your question here'")

## Optional: Save to Google Drive

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Save checkpoints and reports
!mkdir -p /content/drive/MyDrive/nanochat_outputs
!cp -r /content/.cache/nanochat/* /content/drive/MyDrive/nanochat_outputs/ 2>/dev/null || true
!cp report.md /content/drive/MyDrive/nanochat_outputs/ 2>/dev/null || true
print("✅ Saved to Google Drive!")